# 라이브러리

In [1]:
! pip3 install koreanize_matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 19.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib

# 정의

📊 퍼널 단계 정의 (채용 플랫폼 기준)
1. 랜딩 단계 (Landing / 탐색 진입)
    - 관련 URL: home, search, suggest, jobs, companies, people
    - 목적: 플랫폼에 들어와서 정보를 둘러보기 시작함.

2. 정보 탐색 단계 (View Details)
    - 관련 URL: jobs/job_id, companies/company_id, people/id, jobs/id/other_jobs
    - 목적: 개별 채용공고나 기업/인물의 상세 정보 열람

3. 행동 유도 단계 (Engagement / Action Intent)
    - 관련 URL: continue, signup, email_verify, verify_phone, setting
    - 목적: 회원가입을 시작하거나, 지원을 위한 절차 진행

4. 지원 단계 (Apply)
    - 관련 URL: apply, applications/job_id, @user_id?action=request_approval_complete
    - 목적: 실제 채용공고에 지원하는 행동

5. 완료/성과 단계 (Success / Conversion)
    - 관련 URL: password_reset, help, zip_code, admin, app, pricing
    - 목적: 전환 이후 지속 이용 또는 추가 행동을 보임

🔁 전환/이탈률 분석 설계
각 단계의 유입 사용자 수를 기반으로:

|퍼널 단계|방문자 수|전환률 (%)|이탈률 (%)|
|---|---|---|---|
|랜딩|A|-|((A - B) / A) * 100|
|정보 탐색|B|(B / A) * 100|((B - C) / B) * 100|
|행동 유도|C|(C / B) * 100|((C - D) / C) * 100|
|지원|D|(D / C) * 100|((D - E) / D) * 100|
|전환 완료|E|(E / D) * 100|-|

# 데이터 불러오기

In [3]:
log_2022_path = "/content/drive/MyDrive/코드잇_데이터분석_6기/프로젝트/데이터 분석 중급 프로젝트_1/원본 데이터/주제 1. 국내 채용시장 및 채용 플랫폼 이용패턴 분석/log_2022.csv"
log_2022_df = pd.read_csv(log_2022_path)
log_2022_df.head(2)

,Unnamed: 0,user_uuid,URL,timestamp,date,response_code,method
0,0,8a7e70cd-1630-4a8a-8303-0706af3a7908,api/search/language?name=한국어&_=1655915651225,2022-06-22 16:42:48.247454 UTC,2022-06-23,200,GET
1,1,8a7e70cd-1630-4a8a-8303-0706af3a7908,api/signup/form,2022-06-22 16:19:07.770741 UTC,2022-06-23,200,POST


In [4]:
log_2023_path = "/content/drive/MyDrive/코드잇_데이터분석_6기/프로젝트/데이터 분석 중급 프로젝트_1/원본 데이터/주제 1. 국내 채용시장 및 채용 플랫폼 이용패턴 분석/log_2023.csv"
log_2023_df = pd.read_csv(log_2023_path)
log_2023_df.head(2)

,Unnamed: 0,user_uuid,URL,timestamp,date,response_code,method
0,0,5ce8f5ca-3476-4623-a60c-00c98eef3b62,@user_id,2023-12-29 13:19:50.230356 UTC,2023-12-29,200,GET
1,1,5ce8f5ca-3476-4623-a60c-00c98eef3b62,api/users/notifications/mark_read?id=6425064&_...,2023-12-29 13:20:17.848762 UTC,2023-12-29,200,GET


In [5]:
# 1. UTC 제거
log_2022_df["timestamp_clean"] = log_2022_df["timestamp"].str.replace(" UTC", "", regex=False)

# 2. datetime으로 변환
log_2022_df["timestamp_clean"] = pd.to_datetime(
    log_2022_df["timestamp_clean"],
    format='%Y-%m-%d %H:%M:%S.%f',
    errors='coerce'
)

# 3. NaT가 생긴 경우, 포맷이 다른 문자열을 다시 처리 (마이크로초 없는 경우 등)
mask_failed = log_2022_df["timestamp_clean"].isna()
log_2022_df.loc[mask_failed, "timestamp_clean"] = pd.to_datetime(
    log_2022_df.loc[mask_failed, "timestamp"].str.replace(" UTC", "", regex=False),
    format="%Y-%m-%d %H:%M:%S",
    errors="coerce"
)

# 4. 정리
log_2022_df["timestamp"] = log_2022_df["timestamp_clean"]
log_2022_df.drop(columns="timestamp_clean", inplace=True)

In [6]:
# 1. UTC 제거
log_2023_df["timestamp_clean"] = log_2023_df["timestamp"].str.replace(" UTC", "", regex=False)

# 2. datetime으로 변환
log_2023_df["timestamp_clean"] = pd.to_datetime(
    log_2023_df["timestamp_clean"],
    format='%Y-%m-%d %H:%M:%S.%f',
    errors='coerce'
)

# 3. NaT가 생긴 경우, 포맷이 다른 문자열을 다시 처리 (마이크로초 없는 경우 등)
mask_failed = log_2023_df["timestamp_clean"].isna()
log_2023_df.loc[mask_failed, "timestamp_clean"] = pd.to_datetime(
    log_2023_df.loc[mask_failed, "timestamp"].str.replace(" UTC", "", regex=False),
    format="%Y-%m-%d %H:%M:%S",
    errors="coerce"
)

# 4. 정리
log_2023_df["timestamp"] = log_2023_df["timestamp_clean"]
log_2023_df.drop(columns="timestamp_clean", inplace=True)

In [ ]:
# UTM 파라미터 확인
log_2022_df[log_2022_df["URL"].str.contains("utm_", case=False, na=False)]

,Unnamed: 0,user_uuid,URL,timestamp,date,response_code,method
20,20,8a7e70cd-1630-4a8a-8303-0706af3a7908,@user_id?utm_source=notification&utm_medium=em...,2022-06-23 16:35:25.501568,2022-06-24,200,GET
129,129,8a7e70cd-1630-4a8a-8303-0706af3a7908,r?type=experience_verify&c=87c2170d-c7e5-4bb8-...,2022-06-22 16:26:09.919652,2022-06-23,302,GET
168,168,8a7e70cd-1630-4a8a-8303-0706af3a7908,jobs/id/id_title?utm_campaign=google_jobs_appl...,2022-06-22 16:21:07.392774,2022-06-23,200,GET
255,255,207a8876-2a6c-4137-9347-476ce719c541,@user_id?utm_source=notification&utm_medium=em...,2022-06-22 22:10:33.194900,2022-06-23,200,GET
287,287,207a8876-2a6c-4137-9347-476ce719c541,@user_id?utm_source=notification&utm_medium=em...,2022-06-02 22:14:13.023414,2022-06-03,200,GET
...,...,...,...,...,...,...,...
10054105,10054105,d491cf04-1408-4f1a-bc14-3dccccdc05a4,setting?utm_source=notification,2022-08-19 14:45:52.762488,2022-08-19,200,GET
10054107,10054107,b26bf8fc-58d6-4a7c-8dfa-b05c37bc3570,setting?utm_source=notification&utm_medium=ema...,2022-07-11 18:19:19.566660,2022-07-12,200,GET
10054113,10054113,a4295723-f94c-4f3a-8a48-37c1b8e4850a,jobs/id/id_title?utm_campaign=google_jobs_appl...,2022-02-10 06:55:11.196278,2022-02-10,200,GET
10054118,10054118,d2b90cda-7e0d-454f-8a64-070048551e14,setting?utm_source=notification&utm_medium=ema...,2022-07-27 01:47:23.149828,2022-07-27,200,GET


In [ ]:
# UTM 파라미터 제거
from urllib.parse import urlparse

def remove_query_params(url):
    try:
        return urlparse(url).path.lower()
    except:
        return str(url).lower()

log_2022_df["clean_url"] = log_2022_df["URL"].apply(remove_query_params)

# 퍼널 단계 부여

In [ ]:
# clean_url 만들기
log_2022_df["clean_url"] = log_2022_df["clean_url"].str.lower()

# 퍼널 단계 정의
funnel_stages = {
    "landing": ["home", "search", "suggest", "jobs", "companies", "people", "timeline", "discover", "recommend_specialty", "recommend_talents"],
    "view_details": ["job", "company", "people", "other_jobs", "preview", "view", "detail", "post"],
    "engagement": ["continue", "signup", "email_verify", "verify_phone", "setting", "settings", "verify", "step1", "step2", "step3", "references"],
    "apply": ["apply", "applications", "request_approval_complete", "@user_id", "form", "resume", "submit", "project", "experience", "career", "education"],
    "converted": ["password_reset", "help", "zip_code", "admin", "app", "pricing", "benefits", "settings", "profile", "done", "success"]
}

# 퍼널 단계 부여 함수
def assign_funnel_stage(url):
    url = str(url).lower()
    for stage, keywords in funnel_stages.items():
        if any(k in url for k in keywords):
            return stage
    return "other"

# funnel_stage 컬럼 생성
log_2022_df["funnel_stage"] = log_2022_df["clean_url"].apply(assign_funnel_stage)

log_2022_df.head()

,Unnamed: 0,user_uuid,URL,timestamp,date,response_code,method,clean_url,funnel_stage
0,0,8a7e70cd-1630-4a8a-8303-0706af3a7908,api/search/language?name=한국어&_=1655915651225,2022-06-22 16:42:48.247454,2022-06-23,200,GET,api/search/language,landing
1,1,8a7e70cd-1630-4a8a-8303-0706af3a7908,api/signup/form,2022-06-22 16:19:07.770741,2022-06-23,200,POST,api/signup/form,engagement
2,2,8a7e70cd-1630-4a8a-8303-0706af3a7908,api/users/id/template,2022-06-22 16:41:54.449837,2022-06-23,200,POST,api/users/id/template,other
3,3,8a7e70cd-1630-4a8a-8303-0706af3a7908,api/users/id/template,2022-06-23 02:53:47.040932,2022-06-23,200,POST,api/users/id/template,other
4,4,8a7e70cd-1630-4a8a-8303-0706af3a7908,api/users/id/phone/verify/template?_=165591488...,2022-06-22 16:21:58.154299,2022-06-23,200,GET,api/users/id/phone/verify/template,engagement


## other 단계 제외

In [ ]:
# other 행만 모아 새로운 데이터프레임 생성
out_of_stage_df = log_2022_df[log_2022_df["funnel_stage"] == "other"]
out_of_stage_df.head()

,Unnamed: 0,user_uuid,URL,timestamp,date,response_code,method,clean_url,funnel_stage
2,2,8a7e70cd-1630-4a8a-8303-0706af3a7908,api/users/id/template,2022-06-22 16:41:54.449837,2022-06-23,200,POST,api/users/id/template,other
3,3,8a7e70cd-1630-4a8a-8303-0706af3a7908,api/users/id/template,2022-06-23 02:53:47.040932,2022-06-23,200,POST,api/users/id/template,other
9,9,8a7e70cd-1630-4a8a-8303-0706af3a7908,api/users/notifications/mark_read?id=5201462&_...,2022-06-23 02:53:43.702431,2022-06-23,200,GET,api/users/notifications/mark_read,other
11,11,8a7e70cd-1630-4a8a-8303-0706af3a7908,NaN,2022-06-26 08:11:12.699045,2022-06-26,200,GET,nan,other
15,15,8a7e70cd-1630-4a8a-8303-0706af3a7908,api/users/id/language,2022-06-23 16:39:27.054632,2022-06-24,200,PUT,api/users/id/language,other


In [ ]:
# other 행을 제외한 나머지
log_df = log_2022_df[log_2022_df["funnel_stage"] != "other"]
log_df.head()

,Unnamed: 0,user_uuid,URL,timestamp,date,response_code,method,clean_url,funnel_stage
0,0,8a7e70cd-1630-4a8a-8303-0706af3a7908,api/search/language?name=한국어&_=1655915651225,2022-06-22 16:42:48.247454,2022-06-23,200,GET,api/search/language,landing
1,1,8a7e70cd-1630-4a8a-8303-0706af3a7908,api/signup/form,2022-06-22 16:19:07.770741,2022-06-23,200,POST,api/signup/form,engagement
4,4,8a7e70cd-1630-4a8a-8303-0706af3a7908,api/users/id/phone/verify/template?_=165591488...,2022-06-22 16:21:58.154299,2022-06-23,200,GET,api/users/id/phone/verify/template,engagement
5,5,8a7e70cd-1630-4a8a-8303-0706af3a7908,api/users/id/education/template?type=apply&_=1...,2022-06-22 16:25:42.416476,2022-06-23,200,GET,api/users/id/education/template,apply
6,6,8a7e70cd-1630-4a8a-8303-0706af3a7908,api/search/people/job_title?name=data en&_=165...,2022-06-22 16:20:10.851840,2022-06-23,200,GET,api/search/people/job_title,landing


# 각 단계별 사용자 수

URL_1만 포함한 결과

|단계|사용자 수|
|---|---|
|landing|15842|
|view_details|5993|
|engagement|10691|
|apply|15747|
|success|7662|

In [ ]:
# 각 퍼널 단계별 고유 사용자 수 계산
funnel_user_counts = log_df.groupby("funnel_stage")["user_uuid"].nunique()

# 순서를 맞춰 정렬
funnel_order = ["landing", "view_details", "engagement", "apply", "converted"]
funnel_user_counts = funnel_user_counts.reindex(funnel_order).fillna(0).astype(int)

# A, B, C, D, E에 할당
A, B, C, D, E = funnel_user_counts.values

# 단계별 사용자 수를 데이터프레임으로 정리
stage_user_counts_df = pd.DataFrame({
    "단계": ["landing", "view_details", "engagement", "apply", "converted"],
    "사용자 수": [A, B, C, D, E]
})

stage_user_counts_df

,단계,사용자 수
0,landing,16188
1,view_details,9649
2,engagement,11995
3,apply,15861
4,converted,9174


# 전환률 계산

URL_1만 포함한 결과

|단계|방문자 수|전환률 (%)|이탈률 (%)|
|---|---|---|---|
|landing|15842|-|62.17|
|view_details|5993|37.83|-78.39|
|engagement|10691|178.39|-47.29|
|apply|15747|147.29|51.34|
|converted|7662|48.66|-|

In [ ]:
funnel_metrics = {
    "단계": ["landing", "view_details", "engagement", "apply", "converted"],
    "방문자 수": [A, B, C, D, E],
    "전환률 (%)": ["-", round((B / A) * 100, 2) if A else 0, round((C / B) * 100, 2) if B else 0, round((D / C) * 100, 2) if C else 0, round((E / D) * 100, 2) if D else 0],
    "이탈률 (%)": [round(((A - B) / A) * 100, 2) if A else "-", round(((B - C) / B) * 100, 2) if B else "-", round(((C - D) / C) * 100, 2) if C else "-", round(((D - E) / D) * 100, 2) if D else "-", "-"]
}

funnel_df = pd.DataFrame(funnel_metrics)
funnel_df

,단계,방문자 수,전환률 (%),이탈률 (%)
0,landing,16188,-,40.39
1,view_details,9649,59.61,-24.31
2,engagement,11995,124.31,-32.23
3,apply,15861,132.23,42.16
4,converted,9174,57.84,-


# 인사이트

|단계|방문자 수|전환률 (%)|이탈률 (%)|해석|
|---|---|---|---|---|
|landing|16,188|-|40.39%|플랫폼에 유입된 전체 방문자 수
|||||이 중 40%가 다음 단계로 넘어가지 않았다는 건, 첫 화면(검색, 홈, 추천 등)에서 바로 이탈한 사용자가 많다는 의미
|||||사용자에게 첫인상에서 매력을 더 줄 필요가 있음|
|view_details|9,649|59.61%|24.31%|유입자 중 약 60%가 실제로 채용 정보/기업 상세 등을 조회했다는 건 꽤 괜찮은 전환|
|||||하지만 약 24%는 이 단계에서 이탈했기 때문에 상세 페이지 UX 개선 여지가 있음|
|engagement|11,995|124.31%|32.23%|전환률이 100%를 넘는 이유는 이전 단계(view_details)를 건너뛴 사용자도 이 단계에 포함되어 있기 때문|
|||||즉, 상세 정보를 보지 않고 직접 지원 준비나 회원가입 단계로 넘어간 사용자가 존재함을 의미|
|apply|15,861|132.23%|42.16%|마찬가지로 전환률 100% 초과는 engagement를 건너뛴 사용자 때문|
|||||매우 많은 사용자가 지원을 실제로 시도하고 있다는 점에서 이 단계는 가장 높은 전환율을 보여주고 있음|
|||||하지만 여기서도 42%가 이탈, 즉 지원을 완료하지 않았다는 점 주목|
|converted|9,174|57.84%|-|이 단계는 지원 완료 혹은 전환 성공한 유저|
|||||apply 단계 대비 약 58%가 전환에 성공|
|||||최종 이탈률은 꽤 높은 편이라 지원 절차 완료까지의 경험 개선이 필요할 수 있음|
|||||예: 파일 업로드, 입력 단계 복잡성 등|

# 시각화

## Sankey Diagram (Plotly)

In [ ]:
import plotly.graph_objects as go

# 사용자별 퍼널 단계 시퀀스 정리
user_funnel = log_df.sort_values(by=["user_uuid", "timestamp"])
user_funnel = user_funnel.groupby("user_uuid")["funnel_stage"].apply(lambda x: list(dict.fromkeys(x)))  # 중복 제거

# 전이 단계 추출
from collections import Counter
transitions = Counter()

for stages in user_funnel:
    for i in range(len(stages)-1):
        transitions[(stages[i], stages[i+1])] += 1

# 고유 단계 리스트
labels = list({s for pair in transitions.keys() for s in pair})
label_index = {k: i for i, k in enumerate(labels)}

# 출발, 도착, 값 정의
source = [label_index[src] for src, tgt in transitions.keys()]
target = [label_index[tgt] for src, tgt in transitions.keys()]
value = list(transitions.values())

# Sankey 그리기
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels
    ),
    link=dict(
        source=source,
        target=target,
        value=value
    ))])

fig.update_layout(title_text="사용자 퍼널 단계 이동 Sankey Diagram", font_size=12)
fig.show()

## Funnel Chart (단계별 전환 시각화)

In [ ]:
import plotly.express as px

stage_user_counts_df = pd.DataFrame({
    "funnel_stage": ["landing", "view_details", "engagement", "apply", "converted"],
    "users": [A, B, C, D, E]
})

fig = px.funnel(stage_user_counts_df, x='users', y='funnel_stage', title="사용자 퍼널 단계별 전환")
fig.show()

# 퍼널 수정

- 퍼널 수정

'project', 'career', 'schools', 'education' 등은 프로필 편집이라 engagement로 이동

'step', 'resume', 'submit', 'apply_progress' 등은 apply 흐름으로 이동

'password_reset', 'notifications', 'mark_read', 'stat', 'widget' 시스템 설정 기능 이용 로그 등 전환 흐름에 맞지 않아 제외

'admin', 'help', 'pricing', 'zip_code' 운영자용, 계정관리, 고객센터 로그 제외

## 퍼널 단계 부여

In [ ]:
# clean_url 만들기
log_2022_df["clean_url"] = log_2022_df["clean_url"].str.lower()

# 퍼널 단계 정의
funnel_stages = {
    "landing": ["home", "search", "suggest", "jobs", "companies", "people", "timeline", "discover", "recommend_specialty", "recommend_talents"],
    "view_details": ["job", "company", "people", "other_jobs", "preview", "view", "detail", "post"],
    "engagement": ["continue", "signup", "email_verify", "verify_phone", "setting", "settings", "verify", "references", "project", "career", "education"],
    "apply": ["apply", "applications", "request_approval_complete", "@user_id", "form", "resume", "submit", "experience", "step1", "step2", "step3"],
    "converted": ["app", "benefits", "settings", "profile", "done", "success"]
}

# 퍼널 단계 부여 함수
def assign_funnel_stage(url):
    url = str(url).lower()
    for stage, keywords in funnel_stages.items():
        if any(k in url for k in keywords):
            return stage
    return "other"

# funnel_stage 컬럼 생성
log_2022_df["funnel_stage"] = log_2022_df["clean_url"].apply(assign_funnel_stage)

log_2022_df.head()

,Unnamed: 0,user_uuid,URL,timestamp,date,response_code,method,clean_url,funnel_stage
0,0,8a7e70cd-1630-4a8a-8303-0706af3a7908,api/search/language?name=한국어&_=1655915651225,2022-06-22 16:42:48.247454,2022-06-23,200,GET,api/search/language,landing
1,1,8a7e70cd-1630-4a8a-8303-0706af3a7908,api/signup/form,2022-06-22 16:19:07.770741,2022-06-23,200,POST,api/signup/form,engagement
2,2,8a7e70cd-1630-4a8a-8303-0706af3a7908,api/users/id/template,2022-06-22 16:41:54.449837,2022-06-23,200,POST,api/users/id/template,other
3,3,8a7e70cd-1630-4a8a-8303-0706af3a7908,api/users/id/template,2022-06-23 02:53:47.040932,2022-06-23,200,POST,api/users/id/template,other
4,4,8a7e70cd-1630-4a8a-8303-0706af3a7908,api/users/id/phone/verify/template?_=165591488...,2022-06-22 16:21:58.154299,2022-06-23,200,GET,api/users/id/phone/verify/template,engagement


### other 단계 제외

In [ ]:
# other 행만 모아 새로운 데이터프레임 생성
out_of_stage_df = log_2022_df[log_2022_df["funnel_stage"] == "other"]
out_of_stage_df.head()

,Unnamed: 0,user_uuid,URL,timestamp,date,response_code,method,clean_url,funnel_stage
2,2,8a7e70cd-1630-4a8a-8303-0706af3a7908,api/users/id/template,2022-06-22 16:41:54.449837,2022-06-23,200,POST,api/users/id/template,other
3,3,8a7e70cd-1630-4a8a-8303-0706af3a7908,api/users/id/template,2022-06-23 02:53:47.040932,2022-06-23,200,POST,api/users/id/template,other
9,9,8a7e70cd-1630-4a8a-8303-0706af3a7908,api/users/notifications/mark_read?id=5201462&_...,2022-06-23 02:53:43.702431,2022-06-23,200,GET,api/users/notifications/mark_read,other
11,11,8a7e70cd-1630-4a8a-8303-0706af3a7908,NaN,2022-06-26 08:11:12.699045,2022-06-26,200,GET,nan,other
15,15,8a7e70cd-1630-4a8a-8303-0706af3a7908,api/users/id/language,2022-06-23 16:39:27.054632,2022-06-24,200,PUT,api/users/id/language,other


In [ ]:
# other 행을 제외한 나머지
log_df = log_2022_df[log_2022_df["funnel_stage"] != "other"]
log_df.head()

,Unnamed: 0,user_uuid,URL,timestamp,date,response_code,method,clean_url,funnel_stage
0,0,8a7e70cd-1630-4a8a-8303-0706af3a7908,api/search/language?name=한국어&_=1655915651225,2022-06-22 16:42:48.247454,2022-06-23,200,GET,api/search/language,landing
1,1,8a7e70cd-1630-4a8a-8303-0706af3a7908,api/signup/form,2022-06-22 16:19:07.770741,2022-06-23,200,POST,api/signup/form,engagement
4,4,8a7e70cd-1630-4a8a-8303-0706af3a7908,api/users/id/phone/verify/template?_=165591488...,2022-06-22 16:21:58.154299,2022-06-23,200,GET,api/users/id/phone/verify/template,engagement
5,5,8a7e70cd-1630-4a8a-8303-0706af3a7908,api/users/id/education/template?type=apply&_=1...,2022-06-22 16:25:42.416476,2022-06-23,200,GET,api/users/id/education/template,engagement
6,6,8a7e70cd-1630-4a8a-8303-0706af3a7908,api/search/people/job_title?name=data en&_=165...,2022-06-22 16:20:10.851840,2022-06-23,200,GET,api/search/people/job_title,landing


## 각 단계별 사용자 수

In [ ]:
# 각 퍼널 단계별 고유 사용자 수 계산
funnel_user_counts = log_df.groupby("funnel_stage")["user_uuid"].nunique()

# 순서를 맞춰 정렬
funnel_order = ["landing", "view_details", "engagement", "apply", "converted"]
funnel_user_counts = funnel_user_counts.reindex(funnel_order).fillna(0).astype(int)

# A, B, C, D, E에 할당
A, B, C, D, E = funnel_user_counts.values

# 단계별 사용자 수를 데이터프레임으로 정리
stage_user_counts_df = pd.DataFrame({
    "단계": ["landing", "view_details", "engagement", "apply", "converted"],
    "사용자 수": [A, B, C, D, E]
})

stage_user_counts_df

,단계,사용자 수
0,landing,16188
1,view_details,9649
2,engagement,12728
3,apply,15817
4,converted,7625


## 전환률 계산

In [ ]:
funnel_metrics = {
    "단계": ["landing", "view_details", "engagement", "apply", "converted"],
    "방문자 수": [A, B, C, D, E],
    "전환률 (%)": ["-", round((B / A) * 100, 2) if A else 0, round((C / B) * 100, 2) if B else 0, round((D / C) * 100, 2) if C else 0, round((E / D) * 100, 2) if D else 0],
    "이탈률 (%)": [round(((A - B) / A) * 100, 2) if A else "-", round(((B - C) / B) * 100, 2) if B else "-", round(((C - D) / C) * 100, 2) if C else "-", round(((D - E) / D) * 100, 2) if D else "-", "-"]
}

funnel_df = pd.DataFrame(funnel_metrics)
funnel_df

,단계,방문자 수,전환률 (%),이탈률 (%)
0,landing,16188,-,40.39
1,view_details,9649,59.61,-31.91
2,engagement,12728,131.91,-24.27
3,apply,15817,124.27,51.79
4,converted,7625,48.21,-


## 시각화

In [ ]:
import plotly.graph_objects as go

# 사용자별 퍼널 단계 시퀀스 정리
user_funnel = log_df.sort_values(by=["user_uuid", "timestamp"])
user_funnel = user_funnel.groupby("user_uuid")["funnel_stage"].apply(lambda x: list(dict.fromkeys(x)))  # 중복 제거

# 전이 단계 추출
from collections import Counter
transitions = Counter()

for stages in user_funnel:
    for i in range(len(stages)-1):
        transitions[(stages[i], stages[i+1])] += 1

# 고유 단계 리스트
labels = list({s for pair in transitions.keys() for s in pair})
label_index = {k: i for i, k in enumerate(labels)}

# 출발, 도착, 값 정의
source = [label_index[src] for src, tgt in transitions.keys()]
target = [label_index[tgt] for src, tgt in transitions.keys()]
value = list(transitions.values())

# Sankey 그리기
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels
    ),
    link=dict(
        source=source,
        target=target,
        value=value
    ))])

fig.update_layout(title_text="사용자 퍼널 단계 이동 Sankey Diagram", font_size=12)
fig.show()

In [ ]:
import plotly.express as px

stage_user_counts_df = pd.DataFrame({
    "funnel_stage": ["landing", "view_details", "engagement", "apply", "converted"],
    "users": [A, B, C, D, E]
})

fig = px.funnel(stage_user_counts_df, x='users', y='funnel_stage', title="사용자 퍼널 단계별 전환")
fig.show()

# 지민님 방식

## 2022

In [7]:
from urllib.parse import urlparse, parse_qs
from collections import Counter

#  URL 의미 기반 태깅 함수
def map_funnel_stage(url):
    url = str(url).lower()

    if pd.isna(url) or url.strip() == '':
        return '기타'

    if 'search' in url or 'keywords' in url or 'suggest' in url or 'q=' in url:
        return '검색'
    elif 'job_title' in url or 'jobs/id' in url:
        return '공고 보기'
    elif 'companies' in url or 'company_id' in url:
        return '기업 보기'
    elif 'template' in url or 'career' in url or 'experience' in url or 'profile' in url:
        return '이력서 작성'
    elif 'apply/step' in url:
        return '지원 시작'
    elif 'applications' in url:
        return '지원 완료'
    elif 'notifications/mark_read' in url:
        return '알림 확인'
    elif '@user_id' in url or 'follower' in url or 'following' in url:
        return '유저 프로필'
    elif 'setting' in url or 'timeline' in url:
        return '설정'
    elif 'signup' in url or 'password_reset' in url:
        return '회원가입'
    else:
        return '기타'

# URL 쿼리 파라미터 추출
def extract_query_keywords(url):
    try:
        parsed = urlparse(str(url))
        query = parse_qs(parsed.query)
        keywords = []
        for k, v in query.items():
            if k in ['q', 'keywords', 'name', 'type', 'job', 'offset']:
                keywords.append(f"{k}={','.join(v)}")
        return '&'.join(keywords)
    except:
        return ''

# 의미 기반 컬럼 생성
log_2022_df['funnel_stage'] = log_2022_df['URL'].apply(map_funnel_stage)
log_2022_df['query_tags'] = log_2022_df['URL'].apply(extract_query_keywords)

# 분석용 테이블 구성
behavior_table_2022 = log_2022_df[['user_uuid', 'timestamp', 'URL', 'funnel_stage', 'query_tags']].sort_values(by=['user_uuid', 'timestamp'])
behavior_table_2022 = behavior_table_2022[
    ~behavior_table_2022['URL'].str.contains('utm', case=False, na=False) &
    ~behavior_table_2022['URL'].str.startswith('api/', na=False)
].reset_index(drop=True)

behavior_table_2022

,user_uuid,timestamp,URL,funnel_stage,query_tags
0,0002535c-eacb-456b-a620-92c917332ba3,2022-01-15 07:44:06.150657,@user_id?action=request_approval_complete,유저 프로필,
1,0002535c-eacb-456b-a620-92c917332ba3,2022-03-01 05:45:33.359728,@user_id?action=request_approval_complete,유저 프로필,
2,0002535c-eacb-456b-a620-92c917332ba3,2022-04-18 15:12:10.181361,@user_id?action=request_approval_complete,유저 프로필,
3,0002535c-eacb-456b-a620-92c917332ba3,2022-04-19 00:50:26.380715,@user_id,유저 프로필,
4,0002535c-eacb-456b-a620-92c917332ba3,2022-05-27 06:59:11.557600,@user_id?action=request_approval_complete,유저 프로필,
...,...,...,...,...,...
4237929,ffff25ca-c1d7-4fc2-891b-b0df92f95092,2022-07-09 08:50:17.814855,companies/company_id,기업 보기,
4237930,ffff25ca-c1d7-4fc2-891b-b0df92f95092,2022-07-09 08:50:39.766286,companies/company_id,기업 보기,
4237931,ffff25ca-c1d7-4fc2-891b-b0df92f95092,2022-07-09 08:50:47.334803,@user_id,유저 프로필,
4237932,ffff25ca-c1d7-4fc2-891b-b0df92f95092,2022-07-09 08:50:50.367690,companies/company_id,기업 보기,


In [10]:
# 퍼널 단계 순서
funnel_order = ["검색", "공고 보기", "이력서 작성", "지원 시작", "지원 완료"]

# 각 단계별 고유 사용자 수 계산
funnel_user_counts = (
    behavior_table_2022.groupby("funnel_stage")["user_uuid"]
    .nunique()
    .reindex(funnel_order)
    .fillna(0)
    .astype(int)
)

# 각 단계 사용자 수를 A~E로 할당
A, B, C, D, E = funnel_user_counts.values

# 전환률과 이탈률 계산
funnel_metrics = {
    "단계": funnel_order,
    "방문자 수": [A, B, C, D, E],
    "전환률 (%)": [
        "-",
        round((B / A) * 100, 2) if A else 0,
        round((C / B) * 100, 2) if B else 0,
        round((D / C) * 100, 2) if C else 0,
        round((E / D) * 100, 2) if D else 0,
    ],
    "이탈률 (%)": [
        round(((A - B) / A) * 100, 2) if A else "-",
        round(((B - C) / B) * 100, 2) if B else "-",
        round(((C - D) / C) * 100, 2) if C else "-",
        round(((D - E) / D) * 100, 2) if D else "-",
        "-"
    ]
}

# 최종 데이터프레임 생성
funnel_df = pd.DataFrame(funnel_metrics)

# 결과 출력
funnel_df

,단계,방문자 수,전환률 (%),이탈률 (%)
0,검색,11071,-,-13.24
1,공고 보기,12537,113.24,43.87
2,이력서 작성,7037,56.13,77.65
3,지원 시작,1573,22.35,-415.58
4,지원 완료,8110,515.58,-


## 2023

In [8]:
from urllib.parse import urlparse, parse_qs
from collections import Counter

#  URL 의미 기반 태깅 함수
def map_funnel_stage(url):
    url = str(url).lower()

    if pd.isna(url) or url.strip() == '':
        return '기타'

    if 'search' in url or 'keywords' in url or 'suggest' in url or 'q=' in url:
        return '검색'
    elif 'job_title' in url or 'jobs/id' in url:
        return '공고 보기'
    elif 'companies' in url or 'company_id' in url:
        return '기업 보기'
    elif 'template' in url or 'career' in url or 'experience' in url or 'profile' in url:
        return '이력서 작성'
    elif 'apply/step' in url:
        return '지원 시작'
    elif 'applications' in url:
        return '지원 완료'
    elif 'notifications/mark_read' in url:
        return '알림 확인'
    elif '@user_id' in url or 'follower' in url or 'following' in url:
        return '유저 프로필'
    elif 'setting' in url or 'timeline' in url:
        return '설정'
    elif 'signup' in url or 'password_reset' in url:
        return '회원가입'
    else:
        return '기타'

# URL 쿼리 파라미터 추출
def extract_query_keywords(url):
    try:
        parsed = urlparse(str(url))
        query = parse_qs(parsed.query)
        keywords = []
        for k, v in query.items():
            if k in ['q', 'keywords', 'name', 'type', 'job', 'offset']:
                keywords.append(f"{k}={','.join(v)}")
        return '&'.join(keywords)
    except:
        return ''

# 의미 기반 컬럼 생성
log_2023_df['funnel_stage'] = log_2023_df['URL'].apply(map_funnel_stage)
log_2023_df['query_tags'] = log_2023_df['URL'].apply(extract_query_keywords)

# 분석용 테이블 구성
behavior_table_2023 = log_2023_df[['user_uuid', 'timestamp', 'URL', 'funnel_stage', 'query_tags']].sort_values(by=['user_uuid', 'timestamp'])
behavior_table_2023 = behavior_table_2023[
    ~behavior_table_2023['URL'].str.contains('utm', case=False, na=False) &
    ~behavior_table_2023['URL'].str.startswith('api/', na=False)
].reset_index(drop=True)

behavior_table_2023

,user_uuid,timestamp,URL,funnel_stage,query_tags
0,0002535c-eacb-456b-a620-92c917332ba3,2023-01-22 09:25:37.362475,@user_id,유저 프로필,
1,0002535c-eacb-456b-a620-92c917332ba3,2023-01-25 08:41:13.039711,@user_id,유저 프로필,
2,0002535c-eacb-456b-a620-92c917332ba3,2023-01-25 08:41:19.858353,@user_id,유저 프로필,
3,0002535c-eacb-456b-a620-92c917332ba3,2023-01-25 08:41:28.522585,@user_id,유저 프로필,
4,0002535c-eacb-456b-a620-92c917332ba3,2023-01-25 08:41:33.891682,@user_id,유저 프로필,
...,...,...,...,...,...
2871955,ffff25ca-c1d7-4fc2-891b-b0df92f95092,2023-12-18 15:04:22.251057,@user_id,유저 프로필,
2871956,ffff25ca-c1d7-4fc2-891b-b0df92f95092,2023-12-18 15:05:58.649695,@user_id,유저 프로필,
2871957,ffff25ca-c1d7-4fc2-891b-b0df92f95092,2023-12-18 15:06:35.373638,@user_id,유저 프로필,
2871958,ffff25ca-c1d7-4fc2-891b-b0df92f95092,2023-12-18 15:06:36.763225,@user_id,유저 프로필,


In [13]:
# 유저별 시간 순 정렬 후 중복 제거한 의미 기반 퍼널 경로 추출
user_funnel_path = behavior_table_2023.groupby('user_uuid')['funnel_stage'].apply(
    lambda x: list(pd.Series(x).drop_duplicates())
).reset_index()

# 경로를 문자열로 연결
user_funnel_path['funnel_path_str'] = user_funnel_path['funnel_stage'].apply(lambda x: ' → '.join(x))

# 가장 흔한 의미 기반 행동 흐름 TOP 15 추출
top_funnel_paths = user_funnel_path['funnel_path_str'].value_counts().head(15).reset_index()
top_funnel_paths.columns = ['의미 기반 퍼널 경로', '유저 수']

top_funnel_paths

,의미 기반 퍼널 경로,유저 수
0,기타 → 유저 프로필,585
1,유저 프로필,499
2,기타 → 유저 프로필 → 기업 보기,266
3,기타,196
4,유저 프로필 → 기타,161
5,기업 보기,144
6,유저 프로필 → 기업 보기,97
7,기업 보기 → 유저 프로필,91
8,기타 → 유저 프로필 → 기업 보기 → 공고 보기,88
9,유저 프로필 → 기업 보기 → 기타,87


In [14]:
# 퍼널 순서를 정의 (명시적으로 순서화)
funnel_order = ['검색', '공고 보기', '기업 보기', '이력서 작성', '지원 시작', '지원 완료']

# 유저별 단계 도달 여부를 one-hot encoding 형태로 생성
for stage in funnel_order:
    user_funnel_path[stage] = user_funnel_path['funnel_stage'].apply(lambda x: stage in x)

# 단계별 이탈률 계산
dropoffs = []
for i in range(len(funnel_order) - 1):
    stage_current = funnel_order[i]
    stage_next = funnel_order[i + 1]

    users_in_current = user_funnel_path[user_funnel_path[stage_current] == True]
    users_in_next = users_in_current[users_in_current[stage_next] == True]

    dropoff_rate = 1 - len(users_in_next) / len(users_in_current) if len(users_in_current) > 0 else 0
    dropoffs.append(round(dropoff_rate * 100, 2))

# 결과 DataFrame
funnel_dropoff_df = pd.DataFrame({
    '단계': [f'{funnel_order[i]} → {funnel_order[i+1]}' for i in range(len(funnel_order)-1)],
    '이탈률(%)': dropoffs
})

funnel_dropoff_df

,단계,이탈률(%)
0,검색 → 공고 보기,14.44
1,공고 보기 → 기업 보기,6.98
2,기업 보기 → 이력서 작성,56.87
3,이력서 작성 → 지원 시작,82.76
4,지원 시작 → 지원 완료,7.67


In [11]:
# 퍼널 단계 순서
funnel_order = ["검색", "공고 보기", "이력서 작성", "지원 시작", "지원 완료"]

# 각 단계별 고유 사용자 수 계산
funnel_user_counts = (
    behavior_table_2023.groupby("funnel_stage")["user_uuid"]
    .nunique()
    .reindex(funnel_order)
    .fillna(0)
    .astype(int)
)

# 각 단계 사용자 수를 A~E로 할당
A, B, C, D, E = funnel_user_counts.values

# 전환률과 이탈률 계산
funnel_metrics = {
    "단계": funnel_order,
    "방문자 수": [A, B, C, D, E],
    "전환률 (%)": [
        "-",
        round((B / A) * 100, 2) if A else 0,
        round((C / B) * 100, 2) if B else 0,
        round((D / C) * 100, 2) if C else 0,
        round((E / D) * 100, 2) if D else 0,
    ],
    "이탈률 (%)": [
        round(((A - B) / A) * 100, 2) if A else "-",
        round(((B - C) / B) * 100, 2) if B else "-",
        round(((C - D) / C) * 100, 2) if C else "-",
        round(((D - E) / D) * 100, 2) if D else "-",
        "-"
    ]
}

# 최종 데이터프레임 생성
funnel_df = pd.DataFrame(funnel_metrics)

# 결과 출력
funnel_df

,단계,방문자 수,전환률 (%),이탈률 (%)
0,검색,9275,-,-9.79
1,공고 보기,10183,109.79,46.45
2,이력서 작성,5453,53.55,74.66
3,지원 시작,1382,25.34,-451.88
4,지원 완료,7627,551.88,-


# 파라미터 분석

In [17]:
from urllib.parse import urlparse, parse_qs

# 안전하게 파라미터 추출하는 함수
def extract_query_params(url):
    if isinstance(url, str):
        parsed = urlparse(url)
        query = parse_qs(parsed.query)
        return query if query else None
    return None

# 적용
log_2022_df['params'] = log_2022_df['URL'].apply(extract_query_params)

log_2022_df[['URL', 'params']].head()

,URL,params
0,api/search/language?name=한국어&_=1655915651225,"{'name': ['한국어'], '_': ['1655915651225']}"
1,api/signup/form,None
2,api/users/id/template,None
3,api/users/id/template,None
4,api/users/id/phone/verify/template?_=165591488...,{'_': ['1655914889887']}


In [19]:
# 모든 파라미터 key들을 리스트로 추출
all_keys = log_2022_df['params'].dropna().apply(lambda d: list(d.keys()))

# 리스트 안의 리스트를 평탄화하고 set으로 unique 추출
from itertools import chain
unique_param_keys = set(chain.from_iterable(all_keys))

print(unique_param_keys)

{'sort', 'stock', 'utm_content', 'company', 'standalone', 'hss_channel', 'category', 'amp;utm_campaign', 'specialty', 'utm_medium', 'index', 'finish', 'school', 'sent', 'amp;utm_medium', 'token', 'source', 'obj_id', 'tap_is_link', 'next', 'hiring_types', '_branch_match_id', 'next_url', 'exit_type', 'usg', 'ref', 'remote', 'exclude', 'tag', 'military_service', 'id', 'name', 'expanded', 'keyword', '_changelist_filters', 'order', 'active_offer', 'msclkid', 'job', '_', 'limit', 'q', 'goto', 'size', 'page', 'user_received', 'code', 'rel', 'career', 'action', 'language', 'status', 'specialty_list', 'hl', 'date', 'f', 'fbclid', 'ust', 'filtered', 'traffic_source', 'keywords', 'investment_volume', 'o', 'oneline', '_branch_referrer', 'p', 'utm_term', 'funding_volume', 'related', 'career_type', 'utm_campaign', 'event_context', 'offset', 'edit', 'salary', 'c', 'location', 'utm_source', 'funding', 'type'}
